In [1]:
import os
import numpy as np
import numpy.random  as rd
import scipy as sp
from scipy import stats as st
from collections import Counter
import pandas as pd
import itertools

import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
import matplotlib.animation as ani
import random
plt.rcParams["patch.force_edgecolor"] = True
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]
from IPython.display import display
%matplotlib inline

In [2]:
org_data = pd.read_csv("../data/kddcup.data.corrected")
header_data = pd.read_csv("../data/header.csv")
display(header_data)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate


In [3]:
header = list(header_data.columns)
header.append("label")
org_data.columns = header

In [4]:
display(org_data.head())
display(org_data.tail())
print(org_data.shape)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,238,1282,0,0,0,0,...,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
4898425,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898426,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1098,0,0,0,0,...,255,1.0,0.0,0.14,0.05,0.0,0.01,0.0,0.0,normal.


(4898430, 42)


In [5]:
df = org_data.copy()
display(df.head())

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,238,1282,0,0,0,0,...,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.


In [6]:
dataset = pd.read_csv("../data/SF10.csv")
display(dataset.head())
display(dataset.tail())

,Unnamed: 0,duration,src_bytes,dst_bytes,label,S_L,S_H
0,3429021,1.131402,6.764000,5.790266,normal.,inf,2.222222e+07
1,720878,4.305416,5.338019,7.908056,normal.,46.952558,6.509177e+06
2,299626,-2.302585,5.517854,8.729898,normal.,51.109855,1.315037e+07
3,10954,-2.302585,5.659831,7.838383,normal.,44.277051,1.580741e+07
4,497911,0.095310,6.815750,5.796362,normal.,34.088486,1.725327e+07


,Unnamed: 0,duration,src_bytes,dst_bytes,label,S_L,S_H
99995,4554267,-2.302585,5.620763,5.746523,normal.,10.156763,NaN
99996,6675,-2.302585,5.313698,8.439037,normal.,9.954124,NaN
99997,1368339,-2.302585,5.846728,8.762192,normal.,10.754336,NaN
99998,22059,-2.302585,5.759217,7.526233,normal.,9.649692,NaN
99999,275386,-2.302585,5.720640,7.173268,normal.,9.510338,NaN


In [19]:
ratio = []
attack_tops = []
attack = dataset.loc[dataset.label != "normal."].shape[0]
A = dataset.shape[0] * np.array([0.01, 0.03, 0.05, 0.1, 0.15, 0.2])
for i in A:
    top = dataset.sort("S_L", ascending=False)[:int(i)].copy()
    attack_top = top.loc[top.label != "normal."].shape[0]
    attack_tops.append(attack_top)
    ratio.append(attack_top/attack)
ratio = np.array(ratio)
attack_tops = np.array(attack_tops)

/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [31]:
a = np.concatenate([ratio[:, np.newaxis]*100, attack_tops[:, np.newaxis]], axis=1)
a

array([[  12.47113164,   54.        ],
       [  71.36258661,  309.        ],
       [  78.98383372,  342.        ],
       [  89.6073903 ,  388.        ],
       [  93.99538106,  407.        ],
       [  99.53810624,  431.        ]])

In [32]:
per = [0.01, 0.03, 0.05, 0.1, 0.15, 0.2]
table = pd.DataFrame(a, index=[str(int(i*100)) + "%" for i in per], columns=["per", "n"])
display(table)

,per,n
1%,12.471132,54.0
3%,71.362587,309.0
5%,78.983834,342.0
10%,89.607390,388.0
15%,93.995381,407.0
20%,99.538106,431.0
